# JSONLoader

本 Notebook 提供了使用 JSON [文档加载器](https://python.langchain.com/docs/concepts/document_loaders) 的快速入门指南。有关 JSONLoader 所有功能和配置的详细文档，请参阅[API 参考](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.json_loader.JSONLoader.html)。

- TODO：添加任何其他相关链接，例如有关底层 API 的信息等。

## 概述
### 集成详情

| Class | Package | Local | Serializable | [JS 支持](https://js.langchain.com/docs/integrations/document_loaders/file_loaders/json/)|
| :--- | :--- | :---: | :---: |  :---: |
| [JSONLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.json_loader.JSONLoader.html) | [langchain_community](https://python.langchain.com/api_reference/community/index.html) | ✅ | ❌ | ✅ |
### 加载器功能
| Source | Document Lazy Loading | Native Async Support
| :---: | :---: | :---: |
| JSONLoader | ✅ | ❌ |

## 设置

要访问 JSON 文档加载器，您需要安装 `langchain-community` 集成包以及 ``jq`` Python 包。

### 凭据

使用 `JSONLoader` 类不需要凭据。

要启用模型调用的自动跟踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

安装 **langchain\_community** 和 **jq**:

In [ ]:
%pip install -qU langchain_community jq 

## 初始化

现在，我们可以实例化模型对象并加载文档了：

- TODO：使用相关的参数更新模型实例化。

In [1]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path="./example_data/facebook_chat.json",
    jq_schema=".messages[].content",
    text_content=False,
)

## 加载

In [2]:
docs = loader.load()
docs[0]

Document(metadata={'source': '/Users/isaachershenson/Documents/langchain/docs/docs/integrations/document_loaders/example_data/facebook_chat.json', 'seq_num': 1}, page_content='Bye!')

In [3]:
print(docs[0].metadata)

{'source': '/Users/isaachershenson/Documents/langchain/docs/docs/integrations/document_loaders/example_data/facebook_chat.json', 'seq_num': 1}


## 延迟加载

In [4]:
pages = []
for doc in loader.lazy_load():
    pages.append(doc)
    if len(pages) >= 10:
        # do some paged operation, e.g.
        # index.upsert(pages)

        pages = []

## 从 JSON Lines 文件读取

如果想从 JSON Lines 文件加载文档，请传递 `json_lines=True` 并指定 `jq_schema` 来从单个 JSON 对象中提取 `page_content`。

In [5]:
loader = JSONLoader(
    file_path="./example_data/facebook_chat_messages.jsonl",
    jq_schema=".content",
    text_content=False,
    json_lines=True,
)

docs = loader.load()
print(docs[0])

page_content='Bye!' metadata={'source': '/Users/isaachershenson/Documents/langchain/docs/docs/integrations/document_loaders/example_data/facebook_chat_messages.jsonl', 'seq_num': 1}


## 读取特定内容键

另一种方法是设置 `jq_schema='.'`，并提供 `content_key` 以便仅加载特定内容：

In [6]:
loader = JSONLoader(
    file_path="./example_data/facebook_chat_messages.jsonl",
    jq_schema=".",
    content_key="sender_name",
    json_lines=True,
)

docs = loader.load()
print(docs[0])

page_content='User 2' metadata={'source': '/Users/isaachershenson/Documents/langchain/docs/docs/integrations/document_loaders/example_data/facebook_chat_messages.jsonl', 'seq_num': 1}


## JSON 文件，包含 jq schema `content_key`

要使用 jq schema 中的 `content_key` 加载 JSON 文件中的文档，请将 `is_content_key_jq_parsable` 设置为 `True`。请确保 `content_key` 兼容，并且可以使用 jq schema 进行解析。

In [7]:
loader = JSONLoader(
    file_path="./example_data/facebook_chat.json",
    jq_schema=".messages[]",
    content_key=".content",
    is_content_key_jq_parsable=True,
)

docs = loader.load()
print(docs[0])

page_content='Bye!' metadata={'source': '/Users/isaachershenson/Documents/langchain/docs/docs/integrations/document_loaders/example_data/facebook_chat.json', 'seq_num': 1}


## 提取元数据

通常，我们希望将 JSON 文件中提供的元数据包含在我们从内容创建的文档中。

以下示例演示了如何使用 `JSONLoader` 提取元数据。

有几个关键的变化需要注意。在先前未收集元数据的示例中，我们能够直接在 schema 中指定 `page_content` 的值来源。

在此示例中，我们必须告诉加载器遍历 `messages` 字段中的记录。然后，`jq_schema` 必须是 `.messages[]`。

这使我们能够将记录（dict）传递给必须实现的 `metadata_func`。`metadata_func` 负责识别记录中应包含在最终 `Document` 对象中存储的元数据中的信息片段。

此外，现在我们必须通过 `content_key` 参数在加载器中明确指定，从记录中提取 `page_content` 值所需的键。

In [8]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["sender_name"] = record.get("sender_name")
    metadata["timestamp_ms"] = record.get("timestamp_ms")

    return metadata


loader = JSONLoader(
    file_path="./example_data/facebook_chat.json",
    jq_schema=".messages[]",
    content_key="content",
    metadata_func=metadata_func,
)

docs = loader.load()
print(docs[0].metadata)

{'source': '/Users/isaachershenson/Documents/langchain/docs/docs/integrations/document_loaders/example_data/facebook_chat.json', 'seq_num': 1, 'sender_name': 'User 2', 'timestamp_ms': 1675597571851}


## API 参考

有关 JSONLoader 所有功能和配置的详细文档，请访问 API 参考：https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.json_loader.JSONLoader.html